<a href="https://colab.research.google.com/github/NadiaHolmlund/M6_Group_Assignments/blob/main/Group_Assignment_1/Group_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

Develop a Proof-of-Concept version of an application that is querying a database to come provide an output to the user.

This can be for example:
- Selecting observations from database, performing prediction with a (beforehand fitted) SML model.
- Perform a UML procedure on observations queried from a database.
- Perform a semantic/similarity search for an user input, retrieve most similar docs from a database.

The data used should be non-trivial (eg.: enough observations,´maybe multiple tables, different types of data…)
 - The solution has to be self-contained. This can be done:
 - Within a colab using for grad.io. (Hint: An option is to save the database on github, and then load it in the colab).)
 - As a streamlit app (figure out how to make it self-contained).
 - (sky is the limit.)

Possible databases:
- SQL DB (eg. SQL-lite)
- NoSQL DB
 - Document (eg. tinyDB)
 - Vector (Eg. Faiss, Chroma)

# Solution

In the following, we have created a SQLite database containing information about the 2.000 most cited documents on Scopus within the topic of Natural Language Processing.

Subsequently, a T5 summarization pipeline has been applied from HuggingFace to generate super-summarized abstracts of 50 characters or less. This provides users with a quick overview of the documents' content. The model is demonstrated in Grad.io in which the user can select a document from the Scopus database or upload their own abstract/abstracts not available in the database. The interface then returns a super-summarized abstract.

# Imports

In [3]:
!pip install gradio --q
!pip install transformers --q

In [4]:
import pandas as pd
import sqlite3
from transformers import pipeline
import gradio as gr
from IPython.display import Image

pd.set_option('max_colwidth', 1000)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 1000]


# Creating the database

In [5]:
# Reading the CSV file into a Pandas DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/NadiaHolmlund/M6_Group_Assignments/main/Group_Assignment_1/Data/Scopus_NLP.csv')

In [6]:
# Examining the DataFrame
df.head()

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"Pennington J., Socher R., Manning C.D.",22953926600;24766896100;35280197500;,GloVe: Global vectors for word representation,2014,"EMNLP 2014 - 2014 Conference on Empirical Methods in Natural Language Processing, Proceedings of the Conference",NaN,NaN,NaN,1532,1543,...,9781937284961,NaN,NaN,English,"EMNLP - Conf. Empir. Methods Nat. Lang. Process., Proc. Conf.",Conference Paper,Final,NaN,Scopus,2-s2.0-84961289992
1,"Devlin J., Chang M.-W., Lee K., Toutanova K.",54879967400;25925685700;56349980800;6506107920;,BERT: Pre-training of deep bidirectional transformers for language understanding,2019,NAACL HLT 2019 - 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies - Proceedings of the Conference,1,NaN,NaN,4171,4186,...,9781950737130,NaN,NaN,English,NAACL HLT - Conf. N. Am. Chapter Assoc. Comput. Linguistics: Hum. Lang. Technol. - Proc. Conf.,Conference Paper,Final,NaN,Scopus,2-s2.0-85083815650
2,"Cho K., Van Merriënboer B., Gulcehre C., Bahdanau D., Bougares F., Schwenk H., Bengio Y.",55722769200;57188495900;56006846900;57188434700;42061073000;7005072756;7003958245;,Learning phrase representations using RNN encoder-decoder for statistical machine translation,2014,"EMNLP 2014 - 2014 Conference on Empirical Methods in Natural Language Processing, Proceedings of the Conference",NaN,NaN,NaN,1724,1734,...,9781937284961,NaN,NaN,English,"EMNLP - Conf. Empir. Methods Nat. Lang. Process., Proc. Conf.",Conference Paper,Final,"All Open Access, Green",Scopus,2-s2.0-84961291190
3,"Pang B., Lee L., Vaithyanathan S.",8644537200;7404389769;6603253116;,Thumbs up? Sentiment Classification using Machine Learning Techniques,2002,"Proceedings of the 2002 Conference on Empirical Methods in Natural Language Processing, EMNLP 2002",NaN,NaN,NaN,79,86,...,NaN,NaN,NaN,English,"Proc. Conf. Empir. Methods Nat. Lang. Process., EMNLP",Conference Paper,Final,NaN,Scopus,2-s2.0-85141803251
4,"Collobert R., Weston J., Bottou L., Karlen M., Kavukcuoglu K., Kuksa P.",14064641400;8865128200;6701721644;25651854400;25646533000;57221708009;,Natural language processing (almost) from scratch,2011,Journal of Machine Learning Research,12,NaN,NaN,2493,2537,...,NaN,NaN,NaN,English,J. Mach. Learn. Res.,Article,Final,NaN,Scopus,2-s2.0-80053558787


In [7]:
# Extracting columns to be included in the database
df_clean = df[['Authors', 'Title', 'Abstract']]

In [8]:
# Examining the DataFrame
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Authors   2000 non-null   object
 1   Title     2000 non-null   object
 2   Abstract  2000 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [9]:
# Converting columns from objects to strings
df_clean['Authors'] = df_clean['Authors'].astype(str)
df_clean['Title'] = df_clean['Title'].astype(str)
df_clean['Abstract'] = df_clean['Abstract'].astype(str)

<ipython-input-9-031c1f83abfd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Authors'] = df_clean['Authors'].astype(str)
<ipython-input-9-031c1f83abfd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Title'] = df_clean['Title'].astype(str)
<ipython-input-9-031c1f83abfd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [10]:
# Re-examining the dataframe
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Authors   2000 non-null   object
 1   Title     2000 non-null   object
 2   Abstract  2000 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [11]:
# Creating a connection to the database
conn = sqlite3.connect('Scopus.db')

# Setting up a cursor (pointer to rows in database)
c = conn.cursor()

In [12]:
# Creating a new table for the 'Authors' column
create_authors_query = """
    CREATE TABLE IF NOT EXISTS authors_table (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        author TEXT
    );
"""
conn.execute(create_authors_query)

In [13]:
# Creating a new table for the 'Title' column
create_title_query = """
    CREATE TABLE IF NOT EXISTS title_table (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        author TEXT,
        FOREIGN KEY (author) REFERENCES authors_table (id)
    );
"""
conn.execute(create_title_query)

In [14]:
# Creating a new table for the 'Abstract' column
create_abstract_query = """
    CREATE TABLE abstract_table (
    id INTEGER PRIMARY KEY,
    abstract TEXT,
    title TEXT,
    summary TEXT,
    FOREIGN KEY (title) REFERENCES title_table(title)
);
"""
conn.execute(create_abstract_query)

In [15]:
# Inserting data into the 'authors_table'
authors = set()
for author_str in df_clean['Authors']:
    for author in author_str.split(','):
        authors.add(author.strip())
for author in authors:
    insert_author_query = "INSERT OR IGNORE INTO authors_table (author) VALUES (?);"
    conn.execute(insert_author_query, (author,))


In [16]:
# Inserting data into the 'title_table'
select_authors_query = "SELECT * FROM authors_table;"
authors = {row[1]: row[0] for row in conn.execute(select_authors_query)}
for title, author_str in zip(df_clean['Title'], df_clean['Authors']):
    title = title.replace("'", "''")
    insert_title_query = f"INSERT OR IGNORE INTO title_table (title, author) VALUES ('{title}', '{authors[author_str.split(', ')[0]]}');"
    conn.execute(insert_title_query)

In [17]:
# Inserting data into the 'abstract_table'
select_titles_query = "SELECT * FROM title_table;"
titles = {row[1]: row[0] for row in conn.execute(select_titles_query)}
for abstract, title in zip(df_clean['Abstract'], df_clean['Title']):
    insert_abstract_query = "INSERT INTO abstract_table (abstract, title, summary) VALUES (?, ?, '');"
    conn.execute(insert_abstract_query, (abstract, titles[title]))


In [18]:
# Selecting data from the tables to determine if JOIN works across the tables as intended
select_query = """
    SELECT authors_table.author, title_table.title, abstract_table.abstract
    FROM authors_table
    JOIN title_table ON authors_table.id = title_table.author
    JOIN abstract_table ON title_table.id = abstract_table.title
    LIMIT 5;
"""
cursor = conn.execute(select_query)
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)


('Pennington J.', 'GloVe: Global vectors for word representation', 'Recent methods for learning vector space representations of words have succeeded in capturing fine-grained semantic and syntactic regularities using vector arithmetic, but the origin of these regularities has remained opaque. We analyze and make explicit the model properties needed for such regularities to emerge in word vectors. The result is a new global logbilinear regression model that combines the advantages of the two major model families in the literature: global matrix factorization and local context window methods. Our model efficiently leverages statistical information by training only on the nonzero elements in a word-word cooccurrence matrix, rather than on the entire sparse matrix or on individual context windows in a large corpus. The model produces a vector space with meaningful substructure, as evidenced by its performance of 75% on a recent word analogy task. It also outperforms related models on simil

In [19]:
# Printing an overview of the tables
# Quering the schema for each table from the `sqlite_master` table
c.execute("SELECT name, sql FROM sqlite_master WHERE type='table' ORDER BY name")
schemas = c.fetchall()

# Printing the schema for each table
for schema in schemas:
    table_name, table_schema = schema
    print(f"Table: {table_name}\nSchema:\n{table_schema}\n")

Table: abstract_table
Schema:
CREATE TABLE abstract_table (
    id INTEGER PRIMARY KEY,
    abstract TEXT,
    title TEXT,
    summary TEXT,
    FOREIGN KEY (title) REFERENCES title_table(title)
)

Table: authors_table
Schema:
CREATE TABLE authors_table (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        author TEXT
    )

Table: sqlite_sequence
Schema:
CREATE TABLE sqlite_sequence(name,seq)

Table: title_table
Schema:
CREATE TABLE title_table (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        author TEXT,
        FOREIGN KEY (author) REFERENCES authors_table (id)
    )



In [20]:
# Printing information from the authors_table to test the database
c.execute("SELECT * FROM authors_table LIMIT 5")
authors_table = c.fetchall()
for author_name in authors_table:
    print(author_name)

(1, 'Lehar J.')
(2, 'Wu A.')
(3, 'George S.')
(4, 'Zaragoza H.')
(5, 'Herbert C.')


In [21]:
# Printing information from the title_table to test the database
c.execute("SELECT * FROM title_table LIMIT 5")
title_table = c.fetchall()
for title in title_table:
    print(title)

(1, 'GloVe: Global vectors for word representation', '377')
(2, 'BERT: Pre-training of deep bidirectional transformers for language understanding', '5130')
(3, 'Learning phrase representations using RNN encoder-decoder for statistical machine translation', '42')
(4, 'Thumbs up? Sentiment Classification using Machine Learning Techniques', '4380')
(5, 'Natural language processing (almost) from scratch', '1200')


In [22]:
# Printing information from the abstract_table to test the database
c.execute("SELECT * FROM abstract_table LIMIT 5")
abstract_table = c.fetchall()
for abstract in abstract_table:
    print(abstract)

(1, 'Recent methods for learning vector space representations of words have succeeded in capturing fine-grained semantic and syntactic regularities using vector arithmetic, but the origin of these regularities has remained opaque. We analyze and make explicit the model properties needed for such regularities to emerge in word vectors. The result is a new global logbilinear regression model that combines the advantages of the two major model families in the literature: global matrix factorization and local context window methods. Our model efficiently leverages statistical information by training only on the nonzero elements in a word-word cooccurrence matrix, rather than on the entire sparse matrix or on individual context windows in a large corpus. The model produces a vector space with meaningful substructure, as evidenced by its performance of 75% on a recent word analogy task. It also outperforms related models on similarity tasks and named entity recognition. © 2014 Association fo

# Summarization Pipeline using SQLite

In [23]:
# Loading a pre-trained text-summarization model from HuggingFace Transformers
summarizer = pipeline('summarization', model="t5-base", tokenizer="t5-base", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [24]:
# Extracting abstracts for summarization
abstracts = conn.execute('SELECT abstract FROM abstract_table limit 10')

In [25]:
# Iterating over the abstracts and updating the summary for each one
for i, row in enumerate(abstracts):
    # Extracting the text of the current abstract
    abstract = row[0]
    
    # Summarizing the abstract using the pre-trained summarizer
    summary = summarizer(abstract, max_length=50, min_length=0, do_sample=False)[0]['summary_text']
    
    # Updating the 'summary' column in the abstract_table with the summary of the current abstract
    conn.execute('UPDATE abstract_table SET summary = ? WHERE rowid = ?', (summary, i+1))
    
# Commit the changes to the database
conn.commit()

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:745: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [26]:
# Define the SQL query
query = 'SELECT * FROM abstract_table LIMIT 10'

# Execute the query and convert the result to a DataFrame
df_summed = pd.read_sql_query(query, conn)

In [27]:
# Examining the first abstract
df_summed['abstract'][0]

'Recent methods for learning vector space representations of words have succeeded in capturing fine-grained semantic and syntactic regularities using vector arithmetic, but the origin of these regularities has remained opaque. We analyze and make explicit the model properties needed for such regularities to emerge in word vectors. The result is a new global logbilinear regression model that combines the advantages of the two major model families in the literature: global matrix factorization and local context window methods. Our model efficiently leverages statistical information by training only on the nonzero elements in a word-word cooccurrence matrix, rather than on the entire sparse matrix or on individual context windows in a large corpus. The model produces a vector space with meaningful substructure, as evidenced by its performance of 75% on a recent word analogy task. It also outperforms related models on similarity tasks and named entity recognition. © 2014 Association for Co

In [28]:
# Examining the summarization of the first abstract
df_summed['summary'][0]

'a new global logbilinear regression model is developed . it combines the advantages of global matrix factorization and local context window methods . the model produces a vector space with meaningful substructure .'

Based on above, we determine that the model summarizes the abstract to a satisfying extent.

# Grad.io

## Generate a summary of a title available in our Scopus Database

Unfortunately, there is an issue with the Grad.io interface, which causes the output to fail when submitting a Title from the dropdown menu. However, the code functions without the interface, hence we suspect the issue is related to the dropdown function in Grad.io.

The code is therefore presented with examples below for proof-of-concept, followed by the Grad.io interface design.

In [29]:
# Loading a pre-trained text-summarization model from HuggingFace Transformers
summarizer = pipeline('summarization', model="t5-base", tokenizer="t5-base", framework="tf")

# Connecting to the SQLite database
conn = sqlite3.connect('Scopus.db')

# Defining the dropdown options
c = conn.cursor()
c.execute("SELECT DISTINCT title FROM title_table")
dropdown_options = [row[0] for row in c.fetchall()]

# Defining the function for summarization
def summary(selected_option):
    c.execute("SELECT abstract FROM abstract_table WHERE id = (SELECT id FROM title_table WHERE title = ?)", (selected_option,))
    abstract = c.fetchone()[0]
    summary = summarizer(abstract, max_length=50, min_length=0, do_sample=False)
    return summary[0]['summary_text']

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid t

In [30]:
# Printing the first three titles for proof-of-concept purposes
print(dropdown_options[0])
print(dropdown_options[1])
print(dropdown_options[2])

GloVe: Global vectors for word representation
BERT: Pre-training of deep bidirectional transformers for language understanding
Learning phrase representations using RNN encoder-decoder for statistical machine translation


In [31]:
# Printing the first three abstracts for proof-of-concept purposes
c.execute("SELECT * FROM abstract_table LIMIT 3")
abstract_table = c.fetchall()
for abstract in abstract_table:
    print(abstract)

(1, 'Recent methods for learning vector space representations of words have succeeded in capturing fine-grained semantic and syntactic regularities using vector arithmetic, but the origin of these regularities has remained opaque. We analyze and make explicit the model properties needed for such regularities to emerge in word vectors. The result is a new global logbilinear regression model that combines the advantages of the two major model families in the literature: global matrix factorization and local context window methods. Our model efficiently leverages statistical information by training only on the nonzero elements in a word-word cooccurrence matrix, rather than on the entire sparse matrix or on individual context windows in a large corpus. The model produces a vector space with meaningful substructure, as evidenced by its performance of 75% on a recent word analogy task. It also outperforms related models on similarity tasks and named entity recognition. © 2014 Association fo

In [32]:
# Printing the summarization of the first three abstract for proof-of-concept purposes
print(summary(dropdown_options[0]))
print(summary(dropdown_options[1]))
print(summary(dropdown_options[2]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:745: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


a new global logbilinear regression model is developed . it combines the advantages of global matrix factorization and local context window methods . the model produces a vector space with meaningful substructure .
a new language representation model is called BERT, which stands for Bidirectional Encoder Representations from Transformers . the pre-trained model can be fine-tuned with just one additional output layer .
the proposed model consists of two recurrent neural networks . one encodes a sequence of symbols into a fixedlength vector representation . the other decodes the representation into another sequence .


In [33]:
# Unfortunately, this part of the code fails when submitting a title from the dropdown menu. We suspect the issue is related to the Grad.io dropdown function.

# Creating the Gradio interface
demo = gr.Interface(
    title="Generate a summary of a title available in our Scopus Database",
    fn=summary,
    inputs=gr.inputs.Dropdown(choices=dropdown_options, type="value", label="Select a Title from Scopus"),
    outputs=gr.outputs.Textbox(label="Generated Summary"),
)

# Launching the interface
demo.launch()


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [34]:
# Below is the full code for the Grad.io application

# Loading a pre-trained text-summarization model from HuggingFace Transformers
summarizer = pipeline('summarization', model="t5-base", tokenizer="t5-base", framework="tf")

# Connecting to the SQLite database
conn = sqlite3.connect('Scopus.db')

# Defining the dropdown options
c = conn.cursor()
c.execute("SELECT DISTINCT title FROM title_table")
dropdown_options = [row[0] for row in c.fetchall()]

# Defining the function for summarization
def summary(selected_option):
    c.execute("SELECT abstract FROM abstract_table WHERE id = (SELECT id FROM title_table WHERE title = ?)", (selected_option,))
    abstract = c.fetchone()[0]
    summary = summarizer(abstract, max_length=50, min_length=0, do_sample=False)
    return summary[0]['summary_text']

# Creating the Gradio interface
demo = gr.Interface(
    title="Generate a summary of a title available in our Scopus Database",
    fn=summary,
    inputs=gr.inputs.Dropdown(choices=dropdown_options, type="value", label="Select a Title from Scopus"),
    outputs=gr.outputs.Textbox(label="Generated Summary"),
)

# Launching the interface
demo.launch()

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid t

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Generate a summary of your own abstract/abstracts not available in our Scopus Database

Note that this interface is not connected to the Scopus database, since the input is user-defined.

In [35]:
# Loading a pre-trained text-summarization model from HuggingFace Transformers
summarizer = pipeline('summarization', model="t5-base", tokenizer="t5-base", framework="tf")

# Defining the function for summarization
def summary(abstract):
    summary = summarizer(abstract, max_length=50, min_length=0, do_sample=False)
    return summary[0]['summary_text']

# Defining examples
examples = [
    ["Recent methods for learning vector space representations of words have succeeded in capturing fine-grained semantic and syntactic regularities using vector arithmetic, but the origin of these regularities has remained opaque. We analyze and make explicit the model properties needed for such regularities to emerge in word vectors. The result is a new global logbilinear regression model that combines the advantages of the two major model families in the literature: global matrix factorization and local context window methods. Our model efficiently leverages statistical information by training only on the nonzero elements in a word-word cooccurrence matrix, rather than on the entire sparse matrix or on individual context windows in a large corpus. The model produces a vector space with meaningful substructure, as evidenced by its performance of 75% on a recent word analogy task. It also outperforms related models on similarity tasks and named entity recognition. © 2014 Association for Computational Linguistics."],
    ["We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement). © 2019 Association for Computational Linguistics"],
    ["In this paper, we propose a novel neural network model called RNN Encoder- Decoder that consists of two recurrent neural networks (RNN). One RNN encodes a sequence of symbols into a fixedlength vector representation, and the other decodes the representation into another sequence of symbols. The encoder and decoder of the proposed model are jointly trained to maximize the conditional probability of a target sequence given a source sequence. The performance of a statistical machine translation system is empirically found to improve by using the conditional probabilities of phrase pairs computed by the RNN Encoder-Decoder as an additional feature in the existing log-linear model. Qualitatively, we show that the proposed model learns a semantically and syntactically meaningful representation of linguistic phrases. © 2014 Association for Computational Linguistics."],
]

# Creating the Gradio interface
demo = gr.Interface(
    title="Generate a summary of your own abstract/abstracts not available in our Scopus Database",
    fn=summary,
    inputs=gr.inputs.Textbox(lines=5, label="Input Abstract"),
    outputs=gr.outputs.Textbox(label="Generated Summary"),
    examples=examples
)

# Launching the interface
demo.launch(share=True)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid t

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b6c9af8e889a141e42.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


# Closing the Database Connection

In [36]:
conn.close()